In [1]:
from DNASkittleUtils.Contigs import write_contigs_to_file, read_contigs, Contig
from DNASkittleUtils.DDVUtils import pp

# Jasmine

In [5]:
contigs = read_contigs(r"Ash_Proteome\Jasminum_sambac_exome.fna")
len(contigs)

49772

"Assembly of reads resulted in 113,394 transc ripts and **49,772 unigenes** with mean sizes of 1425  and 846 bp, respectively (Table 2 and Supplementary Material 1)"

divide exome up into 5 pieces

step_size = 49772 // 5 + 6
for start in range(0, len(contigs), step_size):
    write_contigs_to_file('Jasmine%i.fna' % start, contigs[start:start + step_size])

### Translation BioPython

In [2]:
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
coding_dna = Seq("ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG", IUPAC.unambiguous_dna)
coding_dna
# Seq('ATGGCCATTGTAATGGGCCGCTGAAAGGGTGCCCGATAG', IUPACUnambiguousDNA())
str(coding_dna.translate())

'MAIVMGR*KGAR*'

In [7]:
aa_contigs = []
for contig in contigs:
    coding_dna = Seq(contig.seq, IUPAC.unambiguous_dna)
    aa_contigs.append(Contig(contig.name, str(coding_dna.translate())))
write_contigs_to_file('Ash_Proteome/Jasminum_sambac.faa', aa_contigs)


GC_Frame: a:112 t:68 g:29 c:49 
Sequence: aaagccttta ... tcaatatttt, 258 nt, 30.23 %GC


1/1
  S  L  Y  L  Q  V  E  K  Y  Q  K  T  N  S  T  R  K  N  K  P
 K  P  L  P  S  S  *  E  I  S  K  D  K  F  Y  K  E  K  *  T
K  A  F  T  F  K  L  R  N  I  K  R  Q  I  L  Q  G  K  I  N
aaagcctttaccttcaagttgagaaatatcaaaagacaaattctacaaggaaaaataaac   28 %
tttcggaaatggaagttcaactctttatagttttctgtttaagatgttcctttttatttg
A  K  V  K  L  N  L  F  I  L  L  C  I  R  C  P  F  I  F  G 
 F  G  K  G  E  L  Q  S  I  D  F  S  L  N  *  L  S  F  Y  V
  L  R  *  R  *  T  S  F  Y  *  F  V  F  E  V  L  F  F  L  G

61/21
  E  L  Y  Q  K  M  L  Q  V  K  N  A  R  F  I  I  F  P  L  N
 R  T  L  P  K  N  V  T  S  *  K  C  *  V  H  N  F  S  S  K
P  N  F  T  K  K  C  Y  K  L  K  M  L  G  S  *  F  F  L  *
ccgaactttaccaaaaaatgttacaagttaaaaatgctaggttcataatttttcctctaa   28 %
ggcttgaaatggttttttacaatgttcaatttttacgatccaagtattaaaaaggagatt
F  K  V  L  F  H  *  L  N  F  I  S  P  E  Y  N  K  R  *  V 
 R  V  K  G  F  F  T  V  L  *  F  H  *  T

In [14]:
len([contig for contig in contigs if len(contig.seq) % 3 != 0])

33121

That is a very bad sign if the majority of "exons" aren't composed of length 3 codons!  
**Conclusions** Not going to use this data until hearing back from authors.

------------------

# Ash Annotation

In [4]:
with open('Green_Ash_Annotation_Transfer/BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff', 'r') as infile:
    lines = [line for line in infile.readlines() if 'AUGUSTUS	gene' not in line]
    with open('Green_Ash_Annotation_Transfer/scrubbed_BRAKER_RNASeqandEA_augustus_060618.hints.hasevidenceMANUAL.gff', 'w') as outfile:
        outfile.writelines(lines[:34])  # header
        outfile.writelines([line for line in lines if not line.startswith('#')])        

In [4]:
ls = !dir
ls[:7]

[' Volume in drive D is Data',
 ' Volume Serial Number is 7C71-7E80',
 '',
 ' Directory of D:\\josiah\\Documents\\Research\\Thesis - Genome Symmetry\\DNA_Duplications',
 '',
 '2018-09-20  02:57 PM    <DIR>          .',
 '2018-09-20  02:57 PM    <DIR>          ..']

In [1]:

with open(r"D:\josiah\Documents\Research\Colleagues\Maryam\Hymenoscyphus fraxineus\Hymenoscyphus_fraxineus_EIv2.annotation.23.gff3") as infile:
    lines = list(infile.readlines())
len(lines)

131199

In [9]:
genes = [g for g in lines if 'mRNA' in g]
len(genes)

11095

In [3]:
with open(r"D:\josiah\Documents\Research\Colleagues\Maryam\Hymenoscyphus fraxineus\Genes_of_interest.txt") as infile:
    genes_of_interest = infile.read().split()
len(genes_of_interest)

77

In [11]:
annotations = []
for g in genes:
    name = g[g.rfind('Name=') + len('Name='):].strip()
    if name in genes_of_interest:
        annotations.append(g)
len(annotations)

77

In [15]:
annotations = [g.replace('mRNA', 'gene') for g in annotations]
print(annotations[1])

Hymenoscyphus_fraxineus_EIv2_scaffold_0001	EIv2.0	gene	2357419	2359779	.	-	.	ID=HYMFR746836.2.0_000050670.1;Parent=HYMFR746836.2.0_000050670;Name=HYMFR746836.2.0_000050670.1



In [14]:
with open(r"D:\josiah\Documents\Research\Colleagues\Maryam\Hymenoscyphus fraxineus\Hymenoscyphus_fraxineus_genes_of_interest.gff", 'w') as outfile:
    outfile.write(''.join(annotations))


In [5]:
chromosomes = set()
with open(r"D:\josiah\Projects\DDV\DDV\data\Hg38_genes.gtf", 'r') as infile:
    lines = list(infile.readlines())
    for line in lines:
        chromosomes.add(line.split('\t')[0])
len(lines)
chromosomes

{'1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'CHR_HG126_PATCH',
 'CHR_HG1362_PATCH',
 'CHR_HG142_HG150_NOVEL_TEST',
 'CHR_HG151_NOVEL_TEST',
 'CHR_HG1832_PATCH',
 'CHR_HG2021_PATCH',
 'CHR_HG2030_PATCH',
 'CHR_HG2058_PATCH',
 'CHR_HG2062_PATCH',
 'CHR_HG2066_PATCH',
 'CHR_HG2095_PATCH',
 'CHR_HG2104_PATCH',
 'CHR_HG2128_PATCH',
 'CHR_HG2191_PATCH',
 'CHR_HG2217_PATCH',
 'CHR_HG2232_PATCH',
 'CHR_HG2233_PATCH',
 'CHR_HG2247_PATCH',
 'CHR_HG2249_PATCH',
 'CHR_HG2288_HG2289_PATCH',
 'CHR_HG2291_PATCH',
 'CHR_HG986_PATCH',
 'CHR_HSCHR10_1_CTG1',
 'CHR_HSCHR10_1_CTG2',
 'CHR_HSCHR10_1_CTG4',
 'CHR_HSCHR11_1_CTG1_2',
 'CHR_HSCHR11_1_CTG5',
 'CHR_HSCHR11_1_CTG6',
 'CHR_HSCHR11_1_CTG7',
 'CHR_HSCHR11_1_CTG8',
 'CHR_HSCHR11_2_CTG1',
 'CHR_HSCHR11_2_CTG1_1',
 'CHR_HSCHR11_3_CTG1',
 'CHR_HSCHR12_1_CTG1',
 'CHR_HSCHR12_1_CTG2_1',
 'CHR_HSCHR12_2_CTG2',
 'CHR_HSCHR12_2_CTG2_1',
 'CHR_HSCHR12_3_CTG2'

In [6]:
lines[0]

'CHR_HSCHR6_1_CTG3\thavana\tgene\t168820423\t168820656\t.\t+\t.\tgene_id "ENSG00000281421"; gene_version "1"; gene_name "RP11-35J6.1"; gene_source "havana"; gene_biotype "TEC";\n'

In [11]:
def gff_convert(line):
    line = line.replace('; ', ';').replace(' ', '=')
    return line
with open(r"D:\josiah\Projects\DDV\DDV\data\Hg38_genes.gff", 'w') as outfile:
    for line in lines:
        #if line.startswith('MT'):
        outfile.write(gff_convert(line))

In [12]:
with open(r"D:\josiah\Projects\DDV\DDV\data\Hg38_genes.gff", 'r') as infile:
    with open(r"D:\josiah\Projects\DDV\DDV\data\Hg38_genes_chr20.gff", 'w') as outfile:
        for line in infile:
            if line.startswith('20\t'):
                outfile.write(line)

In [14]:
from DNASkittleUtils.Contigs import write_contigs_to_file, read_contigs, Contig
chr20 = read_contigs(r"D:\Genomes\Human\hg38_chr20.fa")[0]
chr20.seq = chr20.seq[:11100000]
write_contigs_to_file(r"D:\josiah\Projects\DDV\DDV\example_data\hg38_chr20_sample.fa", [chr20])

Done writing  1 contigs and 11,160,261bp


In [15]:
with open(r"D:\josiah\Projects\DDV\DDV\data\Hg38_genes_chr20.gff", 'r') as infile:
    with open(r"D:\josiah\Projects\DDV\DDV\example_data\Hg38_genes_chr20_sample.gff", 'w') as outfile:
        for line in infile:
            if int(line.split('\t')[3]) < 11160261:
                   outfile.write(line)

In [16]:
hg38 = read_contigs(r"D:\Genomes\Human\hg38.fa")
[c.name for c in hg38]

['chr1',
 'chr10',
 'chr11',
 'chr11_KI270721v1_random',
 'chr12',
 'chr13',
 'chr14',
 'chr14_GL000009v2_random',
 'chr14_GL000225v1_random',
 'chr14_KI270722v1_random',
 'chr14_GL000194v1_random',
 'chr14_KI270723v1_random',
 'chr14_KI270724v1_random',
 'chr14_KI270725v1_random',
 'chr14_KI270726v1_random',
 'chr15',
 'chr15_KI270727v1_random',
 'chr16',
 'chr16_KI270728v1_random',
 'chr17',
 'chr17_GL000205v2_random',
 'chr17_KI270729v1_random',
 'chr17_KI270730v1_random',
 'chr18',
 'chr19',
 'chr1_KI270706v1_random',
 'chr1_KI270707v1_random',
 'chr1_KI270708v1_random',
 'chr1_KI270709v1_random',
 'chr1_KI270710v1_random',
 'chr1_KI270711v1_random',
 'chr1_KI270712v1_random',
 'chr1_KI270713v1_random',
 'chr1_KI270714v1_random',
 'chr2',
 'chr20',
 'chr21',
 'chr22',
 'chr22_KI270731v1_random',
 'chr22_KI270732v1_random',
 'chr22_KI270733v1_random',
 'chr22_KI270734v1_random',
 'chr22_KI270735v1_random',
 'chr22_KI270736v1_random',
 'chr22_KI270737v1_random',
 'chr22_KI270738v1_ra

In [18]:
chrom = [c.name for c in hg38 if len(c.name) < 6]
' '.join(chrom)

'chr1 chr10 chr11 chr12 chr13 chr14 chr15 chr16 chr17 chr18 chr19 chr2 chr20 chr21 chr22 chr3 chr4 chr5 chr6 chr7 chr8 chr9 chrM chrX chrY'

In [21]:
hg38_chrom = [c for c in hg38 if len(c.name) < 6]
for c in hg38_chrom:
    c.name = c.name.replace('chrM', 'chrMT').replace('chr', '')
write_contigs_to_file(r"D:\Genomes\Human\hg38_chromosome.fa", hg38_chrom)

Done writing  25 contigs and 3,088,286,401bp


In [22]:
[c.name for c in hg38_chrom]

['1',
 '10',
 '11',
 '12',
 '13',
 '14',
 '15',
 '16',
 '17',
 '18',
 '19',
 '2',
 '20',
 '21',
 '22',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 'MT',
 'X',
 'Y']

In [12]:
from DNASkittleUtils.Contigs import write_contigs_to_file, read_contigs, Contig
alignments = read_contigs(r"D:\josiah\Documents\Research\Thesis - Genome Symmetry\DNA_Duplications\Ash_Proteome\Results_Jun25\Orthologues_Jun26\Alignments\OG0028361.fa")

In [13]:
len(alignments)

2

In [14]:
[a.name for a in alignments]

['Slycopersicum_Solyc07g020850.1.1', 'Slycopersicum_Solyc07g020840.1.1']

In [ ]:
28361  # number of "OG" in Orthologues alignment

In [ ]:
24304  # number of gene trees Results_Jun25/Orthologues_Jul04_2/Gene_Trees

In [ ]:
9906 # number of rows in Duplications.csv mentions 24304 OGs  Results_Jun25/Orthologues_Jul04_2

In [7]:
files = !dir Ash_Proteome\Results_Jun25\Orthologues_Jun26\Alignments
len(files)

28372

In [8]:
files[:100]

[' Volume in drive D is Data',
 ' Volume Serial Number is 7C71-7E80',
 '',
 ' Directory of D:\\josiah\\Documents\\Research\\Thesis - Genome Symmetry\\DNA_Duplications\\Ash_Proteome\\Results_Jun25\\Orthologues_Jun26\\Alignments',
 '',
 '2018-08-28  10:03 AM    <DIR>          .',
 '2018-08-28  10:03 AM    <DIR>          ..',
 '2018-06-27  01:14 AM         1,996,588 OG0000000.fa',
 '2018-06-27  01:14 AM           657,826 OG0000001.fa',
 '2018-06-27  01:14 AM           183,682 OG0000002.fa',
 '2018-06-27  01:14 AM         4,273,082 OG0000003.fa',
 '2018-06-27  01:14 AM           117,855 OG0000004.fa',
 '2018-06-27  01:14 AM           759,679 OG0000005.fa',
 '2018-06-27  01:14 AM           650,877 OG0000006.fa',
 '2018-06-27  01:14 AM           641,243 OG0000007.fa',
 '2018-06-27  01:14 AM         1,205,434 OG0000008.fa',
 '2018-06-27  01:14 AM            82,935 OG0000009.fa',
 '2018-06-27  01:14 AM         2,349,602 OG0000010.fa',
 '2018-06-27  01:14 AM           352,585 OG0000011.fa',
 '2